# Логистическая регрессия c применением tf-idf  и pymorphy2
private LB 0.9066 public 0.90211

In [23]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from scipy import sparse as sp
from scipy import sparse
from extractors import Upper, All_upper, All_lower, Word_length, Vow_count, Con_count, Spec_count, Tolower, Pymorphy_predict,Pymorphy_normal
import pymorphy2
input_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
submit  = pd.read_csv('data/sample_submission.csv')
input_data.head()
mysubmit = pd.read_csv('mytest.csv')

In [23]:
# чтобы заработал GridSearch вынести в отдельный фаил
from sklearn.base import BaseEstimator, TransformerMixin
from scipy import sparse
import re
import pymorphy2


class Upper(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None):
        return sparse.csr_matrix([int(t[0].isupper()) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([int(t[0].isupper()) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing

class All_upper(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None):
        return sparse.csr_matrix([int(t.isupper()) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([int(t.isupper()) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing
    
class All_lower(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None):
        return sparse.csr_matrix([int(t.islower()) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([int(t.islower()) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing

class Word_length(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None):
        return sparse.csr_matrix([len(t) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([len(t) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing

class Vow_count(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None):
        return sparse.csr_matrix([len(re.findall('[уеыаоэяию]', t, re.IGNORECASE)) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([len(t) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing

class Con_count(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None):
        return sparse.csr_matrix([len(t) - len(re.findall('[уеыаоэяию]', t, re.IGNORECASE)) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([len(t) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing
    
class Spec_count(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None):
        return sparse.csr_matrix([len(t) - len(re.findall('[- ]', t, re.IGNORECASE)) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([len(t) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing
    
class Tolower(BaseEstimator, TransformerMixin):
    def fit_transform(self, X, y=None):
        return [t.lower()[:-5] for t in X]
    def transform(self, X, y=None):
        return [t.lower()[:-5] for t in X]
    def fit(self, X, y=None):
        return  # generally does nothing
       
class Pymorphy_predict(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
    def fit_transform(self, X, y=None):
        return sparse.csr_matrix([sum([p.score*int('фам' in p.tag.cyr_repr) for p in self.morph.parse(t)]) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([sum([p.score*int('фам' in p.tag.cyr_repr) for p in self.morph.parse(t)]) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing
    def predict_proba(self,X):
        return[ [1-sum([p.score*int('фам' in p.tag.cyr_repr) for p in self.morph.parse(t)]), 
                   sum([p.score*int('фам' in p.tag.cyr_repr) for p in self.morph.parse(t)])] for t in X ] 

class Pymorphy_normal(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
    def fit_transform(self, X, y=None): 
        return sparse.csr_matrix([ int(t in [p.normal_form for p in self.morph.parse(t)]) for t in X]).transpose()
    def transform(self, X, y=None):
        return sparse.csr_matrix([ int(t in [p.normal_form for p in self.morph.parse(t)]) for t in X]).transpose()
    def fit(self, X, y=None):
        return  # generally does nothing


## Pipeline

In [24]:
tfidf_pipe=Pipeline(steps=[
    ('vect', CountVectorizer(analyzer='char_wb', ngram_range=(1,5), encoding='utf8')),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ])
learning_pipe = Pipeline([
    ('get_features', 
    FeatureUnion([
    ('tfidf', tfidf_pipe),
    ('Upper', Upper()),
    ('All_upper', All_upper()),
    ('All_lower', All_lower()),
    ('Word_length', Word_length()),
    ('Vow_count', Vow_count()),
    ('Con_count', Con_count()),
    ('Spec_count', Spec_count()),
    ('pymorphy', Pymorphy_predict()),
    ('pymorphy_norm', Pymorphy_normal())
    ])),
    ('model', LogisticRegression())
    ])

## GradientBoosting

In [35]:
boost = learning_pipe
boost.set_params(
    model = GradientBoostingClassifier(max_depth = 10),
    get_features__tfidf = None
)

Pipeline(memory=None,
     steps=[('get_features', FeatureUnion(n_jobs=1,
       transformer_list=[('tfidf', None), ('Upper', Upper()), ('All_upper', All_upper()), ('All_lower', All_lower()), ('Word_length', Word_length()), ('Vow_count', Vow_count()), ('Con_count', Con_count()), ('Spec_count', Spec_count()), ('pymorphy', Pymo...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])

In [36]:
param_grid = dict()
boost_cv = GridSearchCV(boost, param_grid,n_jobs= 3,scoring = 'roc_auc')
boost_cv.fit(input_data['Word'],input_data['Label'])
boost_cv.cv_results_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\

{'mean_fit_time': array([79.51672308]),
 'mean_score_time': array([18.87221034]),
 'mean_test_score': array([0.74750595]),
 'mean_train_score': array([0.74458066]),
 'params': [{}],
 'rank_test_score': array([1]),
 'split0_test_score': array([0.76034853]),
 'split0_train_score': array([0.74438997]),
 'split1_test_score': array([0.74561339]),
 'split1_train_score': array([0.71706516]),
 'split2_test_score': array([0.73655559]),
 'split2_train_score': array([0.77228684]),
 'std_fit_time': array([0.56462985]),
 'std_score_time': array([0.23471046]),
 'std_test_score': array([0.00980516]),
 'std_train_score': array([0.02254456])}

In [6]:
boost_cv.best_params_

{}

## Logistic regression

In [33]:
logreg = learning_pipe
logreg.set_params(
    model = LogisticRegression(),
    get_features__tfidf = tfidf_pipe
)

Pipeline(memory=None,
     steps=[('get_features', FeatureUnion(n_jobs=1,
       transformer_list=[('tfidf', None), ('Upper', Upper()), ('All_upper', All_upper()), ('All_lower', All_lower()), ('Word_length', Word_length()), ('Vow_count', Vow_count()), ('Con_count', Con_count()), ('Spec_count', Spec_count()), ('pymorphy', Pymo...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [34]:
param_grid = dict(model__penalty = ['l2'],
                  model__dual = [False],
                  model__C = [0.01],
                  model__class_weight = ['balanced']
                 )
logreg_cv = GridSearchCV(logreg, param_grid, n_jobs= 6,scoring = 'roc_auc')
logreg_cv.fit(input_data['Word'],input_data['Label'])
logreg_cv.cv_results_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\

{'mean_fit_time': array([33.91624983]),
 'mean_score_time': array([14.24820518]),
 'mean_test_score': array([0.85902621]),
 'mean_train_score': array([0.85966096]),
 'param_model__C': masked_array(data=[0.01],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_model__class_weight': masked_array(data=['balanced'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_model__dual': masked_array(data=[False],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_model__penalty': masked_array(data=['l2'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'model__C': 0.01,
   'model__class_weight': 'balanced',
   'model__dual': False,
   'model__penalty': 'l2'}],
 'rank_test_score': array([1]),
 'split0_test_score': array([0.86609786]),
 'split0_train_score': array([0.85346007]),
 'split1_test_score': array([0.86966377]),
 'split1_train_score': 

## Soft vote of LogReg and GB

In [27]:
from sklearn.ensemble import VotingClassifier
param_grid = dict()
vote = VotingClassifier([('boost',boost),('logreg',logreg),('pymorphy',Pymorphy_predict())],voting = 'soft',weights=[2,1,0.5])
vote_cv = GridSearchCV(vote, param_grid, n_jobs= 6,scoring = 'roc_auc')
vote_cv.fit(input_data['Word'],input_data['Label'])
vote_cv.cv_results_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\

{'mean_fit_time': array([81.20602616]),
 'mean_score_time': array([51.03237987]),
 'mean_test_score': array([0.83287029]),
 'mean_train_score': array([0.91029423]),
 'params': [{}],
 'rank_test_score': array([1]),
 'split0_test_score': array([0.81980515]),
 'split0_train_score': array([0.90491541]),
 'split1_test_score': array([0.8248275]),
 'split1_train_score': array([0.91234977]),
 'split2_test_score': array([0.85397884]),
 'split2_train_score': array([0.91361753]),
 'std_fit_time': array([1.32173937]),
 'std_score_time': array([1.1891567]),
 'std_test_score': array([0.01506595]),
 'std_train_score': array([0.00383846])}

In [28]:
vote_cv.best_score_

0.8328702909450558

## Fit and Save

In [31]:
fitted_model, answer = fit(logreg,input_data,test_data)
save(answer, 'vote.csv')

In [32]:
def fit(pipe,input_data,test_data):
    pipe.fit(input_data['Word'], input_data['Label'])
    answer = pipe.predict_proba(test_data['Word'])
    return pipe, answer
def save(answer, name):
    my_submit = [('id', range(188920)),
                 ('Prediction', [t[1] for t in answer])
                ]
    sdf = pd.DataFrame.from_items(my_submit)
    sdf.to_csv(name,columns = ['id','Prediction'],index = False)
    sdf.head()

## DATA TRICKS

In [ ]:
#simple answer
predictions = [int(t[0].isupper()) for t in input_data['Word']]
my_submit = [('id', range(188920)),
             ('Prediction', predictions)
            ]
sdf = pd.DataFrame.from_items(my_submit)
sdf.to_csv('mytest2.csv',columns = ['id','Prediction'],index = False)
sdf.head()